#  Regression on House Pricing Dataset: Variable Selection & Regularization
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

[https://www.kaggle.com/harlfoxem/housesalesprediction]

For each house we know 19 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

## TO DO: insert your ID number ("numero di matricola") below

In [ ]:

#put here your ``numero di matricola''
numero_di_matricola =  1110975

In [ ]:
#import all packages needed
%matplotlib inline
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [ ]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

Extract input and output data. We want to predict the price by using othr features (other than id) as input.

In [ ]:
Data = df.values
# m = number of input samples
m = 3164
Y = Data[:m,2]
X = Data[:m,3:]

## Data Pre-Processing

Split the data into training  set of $m_{train}$ samples, validation set of $m_{val}$ samples and a test set of $m_{test}:=m-m_{train}-m_{val}$ samples.

In [ ]:
# Split data into train (50 samples) and test data (the rest)
m_train = 50

m_test = m - m_train 
from sklearn.cross_validation import train_test_split

XtrainOLS, Xtest, YtrainOLS, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)



Standardize the data.

In [ ]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(XtrainOLS)
XtrainOLS = scaler.transform(XtrainOLS)

Xtest = scaler.transform(Xtest)


## Least-Squares Solution

The routine LinearRegression.score(X,y) computes the *Coefficient of determination* $R^2$, defined as:

$$R^2 = 1- \frac{RSS}{TSS}$$

where $RSS$ is the *Residual Sum of Squares* and $TSS$ is the *Total Sum of Square*. Denoting with $\hat{y}_i$ the $i$-th predicted output values, they are so defined:

\begin{align*}
RSS &= \sum_{i=1}^m (y_i - \hat{y}_i)^2\\
TSS &= \sum_{i=1}^m (y_i -\bar{y}_i)^2, \qquad \qquad \bar{y}_i=\frac{1}{m} \sum_{i=1}^m y_i
\end{align*}

In this notebook we will mostly use the coefficient of determination $R^2$ (instead of the RSS) as a measure to compare models and choose tuning parameters.

### TODO 1

Answer the following: are we interested  in models with low $R^2$ or high $R^2$? Why? (max 5 lines)

#### COMPLETE WITH ANSWER:
We are interested in models with high $R^2$: the closer the coefficient of determination is to 1 the better linear regression fits the data with respect to the average. The RSS represents the deviation of the predictions with respect to the true value and the TSS represents the deviation of the mean value to the true value: better predictions (in comparison to the simple average value) achive an RSS higher than TSS, so the fraction between the two is less than 1 and, ideally, goes to 0 (i.e., RSS $= 0$ means that each prediction is equal to the true value). It is possibile to obatin $R^2 < 0$: in that case the model found performs worse than the predictions based on the average.
 



Now compute the Least-Squares estimate using LinearRegression() in Scikit-learn, and print the corresponding score in training and test data.

In [ ]:
# Least-Squares
from sklearn import linear_model 
#OLS is the linear regression model
OLS = linear_model.LinearRegression()

#fit the model on training data
OLS.fit(XtrainOLS, YtrainOLS)

#obtain predictions on training data
Yhat_tr = OLS.predict(XtrainOLS)


#coefficients from the model
b_LS = np.hstack((OLS.intercept_, OLS.coef_))

print "Coefficient of determination on training data:", OLS.score(XtrainOLS,YtrainOLS)
print "Coefficient of determination on test data:", OLS.score(Xtest,Ytest)


### Confidence Intervals

We now compute the confidence interval for each coefficient.

In [ ]:
# Least-Squares: Confidence Intervals
from scipy.stats import t

Xtrain_im_testrcept = np.hstack((np.ones((XtrainOLS.shape[0],1)), XtrainOLS))

#alpha for confidence intervals
alpha = 0.05

#quantile from t-student distribution
tperc = t.ppf(1-alpha/2, m_train-XtrainOLS.shape[1]-1, loc=0, scale=1)
sigma2 = np.linalg.norm(YtrainOLS-Yhat_tr)**2/(m_train-XtrainOLS.shape[1]-1)

R = np.dot(Xtrain_im_testrcept.transpose(),Xtrain_im_testrcept)
Ur, Sr, Vr = np.linalg.svd(R, full_matrices=1, compute_uv=1)


Sri = (1/Sr)*(Sr>0)
Sri = Sri*(Sri<1e10)

print Sri

Ri2 = np.dot(Ur,np.dot(np.diag(Sri),np.transpose(Ur)))

Ri = np.linalg.pinv(R)

# TO AVOID LATER COMPUTATION OF SQRT OF NEGATIVE VALUES
# Ri_diag = np.diag(Ri).copy()
# Ri_diag[np.where(Rid<0)] = 0
# print Ri_diag
print np.diag(Ri)
print np.diag(Ri2)

# TO AVOID COMPUTATION OF SQRT OF NEGATIVE VALUES
# v = np.sqrt(Ri_diag)
v = np.sqrt(np.diag(Ri))
Delta = np.sqrt(sigma2)*v*tperc
CI = np.transpose(np.vstack((b_LS,b_LS))) + np.transpose(np.vstack((-Delta,+Delta) ))

Plot the LS coefficients and their confidence im_testrval.

In [ ]:
# Plot confidence im_testrvals
plt.figure(1)
plt.plot(b_LS[1:], 'r', marker='o', ms=7.0)
plt.plot(CI[1:,0], 'b--')
plt.plot(CI[1:,1], 'b--')
plt.plot(np.zeros(b_LS.shape[0],), 'k', linewidth=2.0)
plt.xlabel('Coefficient Index')
plt.ylabel('LS Coefficient')
plt.title('Coefficients and Confidence Sets')
plt.show()

### Question: based on the results above, if you had to choose at most 5 features for a linear regression model, which ones would you choose? Why?

### TODO 3
Answer the question above (max 5 lines)

If I had to choose at most 5 features for a linear regression model, based on the results above, I would choose the coefficients that have the highest (absolute) value, thus discarding those closer to 0 since they do not add that much to the model. In this case I can do that because we first normalized the data, so we do not bump into scaling problems. In general we have to take into account scaling problems and also correlations between features before selecting only some of them.

## Best-Subset Selection

Splitting the OLS training data into a training data and validation dataset perform best-subset selection. 

For $k$ going from 1 to $n_{sub}=4$:
1. Compute the LS estimate using all the possible subsets of $k$ features
2. Compute the prediction error on the validation dataset

Finally we choose the subset of $k^*$ features givein the lowest validation error.


In [ ]:
import itertools
import math 

m_trainBSS=int(math.ceil(m_train/2))
m_valBSS=m_train-m_trainBSS


Xtrain = XtrainOLS[:m_trainBSS,:]
Ytrain = YtrainOLS[:m_trainBSS]
Xval = XtrainOLS[m_trainBSS:,:]
Yval = YtrainOLS[m_trainBSS:,]


nsub = 4
features_idx_dict = {}
validation_err_dict = {}
validation_err_min = np.zeros(nsub,)
validation_err_min_idx = np.zeros(nsub, dtype=np.int64)
for k in range(1,nsub+1):
    features_idx = list(itertools.combinations(range(Xtrain.shape[1]),k))
    validation_error = np.zeros(len(features_idx),)
    for j in range(len(features_idx)):
        OLS_subset = linear_model.LinearRegression()
        OLS_subset.fit(Xtrain[:,features_idx[j]], Ytrain)
        validation_error[j] = 1 - OLS_subset.score(Xval[:,features_idx[j]], Yval)
        validation_error[j] = 1 - OLS_subset.score(Xtest[:,features_idx[j]], Ytest)
    validation_err_min[k-1] = np.min(validation_error)    
    validation_err_min_idx[k-1] = np.argmin(validation_error)
    features_idx_dict.update({k: features_idx})
    validation_err_dict.update({k: validation_error})

Plot the validation error as a function of the number of retained features.

In [ ]:
# Plot
plt.figure(2)
for k in range(1,nsub+1):
    plt.scatter(k*np.ones(validation_err_dict[k].shape), validation_err_dict[k], color='k', alpha=0.5)
    #plt.scatter(k, validation_err_min[k-1], color='r', alpha=0.8)
    if k > 1:
        plt.plot([k-1, k], [validation_err_min[k-2], validation_err_min[k-1]], color='r',marker='o', 
            markeredgecolor='k', markerfacecolor = 'r', markersize = 10)
plt.xlabel('Number of retained features')
plt.ylabel('RSS/TSS')
plt.title('Best-Subset Selection')
plt.show()

Compute the LS estimate using the selected subset of features.

### TODO 4: pick the number of features for the best subset according to figure above, select the best subset using the results above and learn the model on the entire training data; finally compute score on training and on test data

In [ ]:
OLS_best_subset = linear_model.LinearRegression()

# now pick the number of features according to best subset
opt_num_features = np.argmin(validation_err_min)

#opt_features_idx contains the indices of the features from best subset
opt_features_idx = features_idx_dict[opt_num_features][validation_err_min_idx[opt_num_features-1]]


#let's print the indices of the features from best subset
print opt_features_idx

#fit the best subset on the entire training set
OLS_best_subset.fit(XtrainOLS[:,opt_features_idx], YtrainOLS)

#print the coefficient of determination on training and on test data
print "Coefficient of determination on training data:", OLS_best_subset.score(XtrainOLS[:,opt_features_idx],YtrainOLS)
print "Coefficient of determination on test data:", OLS_best_subset.score(Xtest[:,opt_features_idx],Ytest)

### TODO 5: do the features from best subset selection correspond to the ones you would have chosen based on confidence im_testrvals for the linear regression coefficients? Comment (max 5 lines)

### ANSWER
No, not entirely, I would not have chosen features number 11, for example, since it has a coefficient with a very low (absolute) value. With 4 features, I would have chosen features 3, 8, 15 and 16 for the reason explained earlier. However, the features selected by Best-Subset Selection (square footage of the home, if the house has been viewed, the overall grade and the year that the house was build) are more meaningful in determining the price of the house than what I would have chosen looking only at value of the features instead of evalutaing also the meaning of such features.

## Lasso

### TO DO 6
Use the routine *lasso_path* from *sklearn.linear_regression* to compute the "lasso path" for different values of the regularization parameter $\lambda$. You should first fix a grid a possible values of lambda (the variable "lasso_lams"). For each entry of the vector "lasso_lams" you should compute the corresponding model (The i-th column of the vector  "lasso_coefs" should contain the coefficients of the linear model computed using lasso_lams[i] as regularization parameter).

Be careful that the grid should be chosen appropriately.
Note that the parameter $\lambda$ is called $\alpha$ in the Lasso model from sklearn.


In [ ]:
from sklearn.linear_model import lasso_path

# select a grid of possible regularization parameters 
# (be carefull how this is chosen, you may have to refine the choice after having seen the results)
lasso_lams = np.arange(1000, 100000, 1000)


# Use the function lasso_path to compute the "lasso path", passing in input the lambda values
# you have specified in lasso_lams
lasso_lams, lasso_coefs, _ = lasso_path(X, Y, alphas=lasso_lams)



Evaluate the sparsity in the estimated coefficients as a function of the regularization parameter $\lambda$: to this purpose, compute the number of non-zero entries in the estimated coefficient vector.

In [ ]:
l0_coef_norm = np.zeros(len(lasso_lams),)

for i in range(len(lasso_lams)):
    l0_coef_norm[i] = sum(lasso_coefs[:,i]!=0)


plt.figure(6)
plt.plot(lasso_lams, l0_coef_norm, marker='o', markersize=5)
plt.xlabel('Lambda')
plt.ylabel('Number of non-zero coefficients')
plt.title('Sparsity Degree')
plt.show()

### TODO 7: explain the results in the figure above (max 5 lines)

### ANSWER
In the figure above we can see how many coefficients are set to 0 on different values of lambda. We note that, in general, the higher the value of lambda is the higher is the number of coefficients set 0. However, the sparsity degree of the coefficients set to 0 with respect to the value of lambda clearly is not a monotonically decreasing function. With this information we can practically perform feature selection using lambda as a way to exclude (i.e., set to zero) features.

### TODO 8: Use k-fold Cross-Validation to fix the regularization parameter

Use the scikit-learn built-in routine *Lasso* (from the *linear_regression* package) to compute the lasso  coefficients.

Use *KFold* from *sklearn.cross_validation* to split the data i.e. XtrainOLS and YtrainOLS) into the desired number of folds.

The pick $lam\_opt$ to be the chosen value for the regularization parameter.

In [ ]:
from sklearn.cross_validation import KFold
num_folds = 5

kf = KFold(n=m_train, n_folds = num_folds)

#loss_ridge_kfold will contain the value of the loss
loss_lasso_kfold = np.zeros(len(lasso_lams),)

for i in range(len(lasso_lams)):
    
    #define a lasso model  using linear_model.Lasso() for the i-th value of lam_values
    lasso_kfold = linear_model.Lasso(alpha=lasso_lams[i]) 
    for train_index, validation_index in kf:
        Xtrain_kfold, Xva_kfold = XtrainOLS[train_index], XtrainOLS[validation_index]
        Ytrain_kfold, Yva_kfold = YtrainOLS[train_index], YtrainOLS[validation_index]
        
        #learn the model using the training data from the k-fold
        lasso_kfold.fit(Xtrain_kfold, Ytrain_kfold)
        
        #compute the loss (loss_lasso_kfold) using the validation data from the k-fold
        loss_lasso_kfold[i] = lasso_kfold.score(Xva_kfold, Yva_kfold)

#choose the regularization parameter that minimizes the loss
lasso_lam_opt =  lasso_lams[np.argmax(loss_lasso_kfold)]

print "Best value of the regularization parameter:", lasso_lam_opt

Plot the Cross-Validation estimate of the prediction error as a function of the regularization parameter

In [ ]:
plt.figure(4)
plt.xscale('log')
plt.plot(lasso_lams, loss_lasso_kfold, color='b')
plt.scatter(lasso_lams[np.argmin(loss_lasso_kfold)], loss_lasso_kfold[np.argmin(loss_lasso_kfold)], color='b', marker='o', linewidths=5)
plt.xlabel('Lambda')
plt.ylabel('Validation Error')
plt.title('Lasso: choice of regularization parameter')
plt.show()
print "Total number of coefficients:", len(lasso_kfold.coef_)
print "Number of non-zero coefficients:", sum(lasso_kfold.coef_ != 0)
print "Best value of regularization parameter:", lasso_lam_opt


### TO DO 9 now estimate the lasso coefficients using all the training data and the optimal regularization parameter (chosen at previous step)

In [ ]:
# Estimate Lasso  Coefficients with all data (trainval) for the the optimal value lasso_lam_opt of the regularization paramter

#define the model
lasso_reg = linear_model.Lasso(alpha=lasso_lam_opt)

#fit using the training data
lasso_reg.fit(XtrainOLS, YtrainOLS)

print "Coefficient of determination on training data:", lasso_reg.score(XtrainOLS,YtrainOLS)
print "Coefficient of determination on test data:", lasso_reg.score(Xtest,Ytest)

Compare the LS and the lasso  coefficients.

In [ ]:
# Compare LS and lasso coefficients
ind = np.arange(1,len(OLS.coef_)+1)  # the x locations for the groups
width = 0.35       # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind, OLS.coef_, width, color='r')
rects2 = ax.bar(ind + width, lasso_reg.coef_, width, color='y')
ax.legend((rects1[0], rects2[0]), ('LS', 'Lasso'))
plt.xlabel('Coefficient Idx')
plt.ylabel('Coefficient Value')
plt.title('LS and Lasso Coefficient')
plt.show()

In [ ]:
print "Coefficient of determination of LS on test data:", OLS.score(Xtest,Ytest)
print "Coefficient of determination of LS (with subset selection) on test data:", OLS_best_subset.score(Xtest[:,opt_features_idx],Ytest)
print "Coefficient of determination of LASSO on test data:", lasso_reg.score(Xtest,Ytest)

### TODO 14: comment and compare the results obtained by the different methods (max 5 lines)

### ANSWER
We performed regularization with three different method, all of them based on the Least Squares method. The "simple" LS performs very poorly on test data. Least Square with Subset Selection performs better of all three, with very close results in both training and test data. LASSO is the best on training data but lose something on test data, overall still performing better of LS. The figure above compares coefficients of LS with coefficients of LASSO and we can see the shrinkage property of LASSO.

### SUGGESTION (not compulsory): repeat the same as above use different data size, and try to understand the main differences



## Evaluate the performance on the test set

